#### things to make modular
* query expansion
* query expansion result aggregation method
* using clean corpus
* retrieval method (dres/bm25)

#### imports

In [1]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd
from query_expansion import expand_queries_and_qrels
from functions import * 
from collections import defaultdict
import copy

In [2]:
cosqa_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/CoSQA_explanations_query_code.csv'
cosqa_deepseek_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_deepseek_explanations_clean.csv'
cosqa_granite_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_granite_explanations_clean.csv'
cosqa_deepseek_temp_0_qe_path = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/cosqa_queries_expanded_deepseek_temp_0.csv'
cosqa_queries_expanded_granite_temp_0pt5_mmr_2_path='/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/cosqa_queries_expanded_granite_temp_0.5_mmr_2.csv'
cosqa_queries_variants_expanded_deepseek= '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/variants_based_prompting/cosqa_queries_variants_expanded_deepseek.csv'
cosqa_queries_variants_expanded_granite='/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/variants_based_prompting/cosqa_queries_variants_expanded_granite.csv'
csn_queries_expanded_granite_temp_0pt5_mmr_path = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/csn/csn_test_queries_expanded_granite_temp_0.5_mmr.csv'
csn_deepseek_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/output/CSN_deepseek.csv'
csn_granite_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/output/CSN_granite.csv'
csn_code_based_queries_expanded = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/csn/csn_test_queries_expanded_granite_code_based.csv'
csn_test_queries_variants_expanded_granite = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/csn/variants_based_prompting/csn_test_queries_variants_expanded_granite.csv'
csn_test_queries_variants_expanded_deepseek= '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/csn/variants_based_prompting/csn_test_queries_variants_expanded_deepseek.csv'

#### No QE

In [3]:
dataset_name = "cosqa" #CodeSearchNet-python  or cosqa
tasks = load_data(dataset_name)
corpus, queries, qrels = tasks[dataset_name]

in tasks 
data loader init


0it [00:00, ?it/s]

In [4]:
df= pd.read_csv(cosqa_granite_updated_path) # csn_granite_path or cosqa_deepseek_updated_path
corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
tasks[dataset_name] = (corpus, queries, qrels)

#### Query Expansion

In [5]:
# CSN qe variants 
# csn_variants_qe_df = pd.read_csv(csn_test_queries_variants_expanded_deepseek)
# csn_variants_qe_df = csn_variants_qe_df.rename(columns= {'generated_explanation': 'expanded_query'})

# dataset_name = "CodeSearchNet-python"
# tasks = load_data(dataset_name)
# corpus, queries, qrels = tasks[dataset_name]

# df= pd.read_csv(csn_granite_path) # csn_granite_path or cosqa_deepseek_updated_path
# corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
# tasks[dataset_name] = (corpus, queries, qrels)

# # updating queries for qe
# new_queries, new_qrels = expand_queries_and_qrels(csn_variants_qe_df, qrels)
# tasks[dataset_name] = (corpus, new_queries, qrels)


# CSN code based qe
# csn_code_based_qe_df = pd.read_csv(csn_code_based_queries_expanded)
# csn_code_based_qe_df=csn_code_based_qe_df.rename(columns={'explanation': 'expanded_query'})
# csn_code_based_qe_df=csn_code_based_qe_df.rename(columns={'id': 'v_id'})
# csn_code_based_qe_df=csn_code_based_qe_df.rename(columns={'query_id': 'id'})

# dataset_name = "CodeSearchNet-python"
# tasks = load_data(dataset_name)
# corpus, queries, qrels = tasks[dataset_name]

# df= pd.read_csv(csn_granite_path) # csn_granite_path or cosqa_deepseek_updated_path
# corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
# tasks[dataset_name] = (corpus, queries, qrels)

# # updating queries for qe
# new_queries, new_qrels = expand_queries_and_qrels(csn_code_based_qe_df, qrels)
# tasks[dataset_name] = (corpus, new_queries, qrels)

# CSN
# csn_qe_df = pd.read_csv(csn_queries_expanded_granite_temp_0pt5_mmr_path)
# csn_qe_df=csn_qe_df.rename(columns={'mmr_explanation': 'expanded_query'})

# dataset_name = "CodeSearchNet-python"
# tasks = load_data(dataset_name)
# corpus, queries, qrels = tasks[dataset_name]

# df= pd.read_csv(csn_granite_path) # csn_granite_path or cosqa_deepseek_updated_path
# corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
# tasks[dataset_name] = (corpus, queries, qrels)

# # updating queries for qe
# new_queries, new_qrels = expand_queries_and_qrels(csn_qe_df, qrels)
# tasks[dataset_name] = (corpus, new_queries, qrels)

# COSQA
# dataset_name = "cosqa"
# tasks = load_data(dataset_name)
# corpus, queries, qrels = tasks[dataset_name]

# # expanded df
# cosqa_qe_df = pd.read_csv(cosqa_queries_variants_expanded_deepseek)
# cosqa_qe_df = cosqa_qe_df.rename(columns={'generated_explanation': 'expanded_query'})
# cosqa_qe_df = cosqa_qe_df[cosqa_qe_df["id"].isin(queries.keys())]


# # updating the corpus with no comments in code
# df= pd.read_csv(cosqa_granite_updated_path)
# corpus = update_corpus_with_cleaned_code(corpus, df) 

# # updating queries for qe
# new_queries, new_qrels = expand_queries_and_qrels(cosqa_qe_df, qrels)
# tasks[dataset_name] = (corpus, new_queries, qrels)

#### Explanations

In [6]:
# if you want to add explanations (deepseek / granite) (cosqa / csn)
tasks_expl = copy.deepcopy(tasks)
#add_expl(tasks_expl, dataset_name, csn_granite_path, "explanation_granite_1") # adding explanation instead of code
#add_expl(tasks, "CodeSearchNet-python", csn_deepseek_path, "explanation_deepseek_1") # adding explanation instead of code
add_expl(tasks_expl, "cosqa", cosqa_granite_updated_path, "explanation_granite_1_cleaned") # adding explanation instead of code

Total docs in corpus after replacement: 4419
Number of docs still missing 'text': 0


#### Running the pipeline

In [7]:
c,b,a = tasks[dataset_name]
f,e,d = tasks_expl[dataset_name]

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed

run_params = [
    run("intfloat/e5-base-v2", tasks, "baseline", "dres", "cosqa/cross-encoder/ms-marco-MiniLM-L-6-v2/0.8"),
    run("BAAI/bge-base-en", tasks, "baseline", "dres", "cosqa/cross-encoder/ms-marco-MiniLM-L-6-v2/0.8"),
    run("intfloat/e5-base-v2", tasks_expl, "granite1", "dres", "cosqa/cross-encoder/ms-marco-MiniLM-L-6-v2/0.8"),
    run("BAAI/bge-base-en", tasks_expl, "granite1", "dres", "cosqa/cross-encoder/ms-marco-MiniLM-L-6-v2/0.8")
    
]

def run_wrapper(params):
    model_name, task, exp_type, retrieval, data_path = params
    return run(model_name, task, exp_type, retrieval, data_path)

# Run all in parallel
if __name__ == "__main__":
    max_workers = 4  # Adjust based on your machine's capacity

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(run_wrapper, params) for params in run_params]

        for future in as_completed(futures):
            try:
                result = future.result()
                print("Run completed successfully.")
            except Exception as e:
                print(f"Run failed: {e}")


YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search codet5.py
encoding queries


Encoding batches:   0%|          | 0/2 [00:00<?, ?batch/s]

encoding corpus


Encoding batches:   0%|          | 0/18 [00:00<?, ?batch/s]

using cross encoder
in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

in combine scores by average
Retrieval evaluation results saved to results/cosqa/cross-encoder/ms-marco-MiniLM-L-6-v2/0.8/baseline/dres/intfloat/e5-base-v2
{'cosqa': {'NDCG': {'NDCG@1': 0.21, 'NDCG@3': 0.28043, 'NDCG@5': 0.3133, 'NDCG@10': 0.34505, 'NDCG@100': 0.40536, 'NDCG@1000': 0.42842}, 'MAP': {'MAP@1': 0.21, 'MAP@3': 0.26267, 'MAP@5': 0.28087, 'MAP@10': 0.29401, 'MAP@100': 0.30474, 'MAP@1000': 0.30555}, 'Recall': {'Recall@1': 0.21, 'Recall@3': 0.332, 'Recall@5': 0.412, 'Recall@10': 0.51, 'Recall@100': 0.81, 'Recall@1000': 0.996}, 'Precision': {'P@1': 0.21, 'P@3': 0.11067, 'P@5': 0.0824, 'P@10': 0.051, 'P@100': 0.0081, 'P@1000': 0.001}}}
YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search codet5.py
encoding queries


...

using cross encoder
in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

in combine scores by average
Retrieval evaluation results saved to results/cosqa/cross-encoder/ms-marco-MiniLM-L-6-v2/0.8/granite1/dres/intfloat/e5-base-v2
{'cosqa': {'NDCG': {'NDCG@1': 0.298, 'NDCG@3': 0.38988, 'NDCG@5': 0.42309, 'NDCG@10': 0.45951, 'NDCG@100': 0.51242, 'NDCG@1000': 0.52579}, 'MAP': {'MAP@1': 0.298, 'MAP@3': 0.36767, 'MAP@5': 0.38577, 'MAP@10': 0.40093, 'MAP@100': 0.41233, 'MAP@1000': 0.41281}, 'Recall': {'Recall@1': 0.298, 'Recall@3': 0.454, 'Recall@5': 0.536, 'Recall@10': 0.648, 'Recall@100': 0.892, 'Recall@1000': 0.998}, 'Precision': {'P@1': 0.298, 'P@3': 0.15133, 'P@5': 0.1072, 'P@10': 0.0648, 'P@100': 0.00892, 'P@1000': 0.001}}}
YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search codet5.py
encoding queries


Encoding batches:   0%|          | 0/2 [00:00<?, ?batch/s]

encoding corpus


Encoding batches:   0%|          | 0/18 [00:00<?, ?batch/s]

using cross encoder
in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

in combine scores by average
Retrieval evaluation results saved to results/cosqa/cross-encoder/ms-marco-MiniLM-L-6-v2/0.8/granite1/dres/BAAI/bge-base-en
{'cosqa': {'NDCG': {'NDCG@1': 0.298, 'NDCG@3': 0.38888, 'NDCG@5': 0.42519, 'NDCG@10': 0.46095, 'NDCG@100': 0.51312, 'NDCG@1000': 0.52689}, 'MAP': {'MAP@1': 0.298, 'MAP@3': 0.367, 'MAP@5': 0.3867, 'MAP@10': 0.40159, 'MAP@100': 0.41284, 'MAP@1000': 0.41336}, 'Recall': {'Recall@1': 0.298, 'Recall@3': 0.452, 'Recall@5': 0.542, 'Recall@10': 0.652, 'Recall@100': 0.892, 'Recall@1000': 1.0}, 'Precision': {'P@1': 0.298, 'P@3': 0.15067, 'P@5': 0.1084, 'P@10': 0.0652, 'P@100': 0.00892, 'P@1000': 0.001}}}
Run failed: cannot unpack non-iterable NoneType object
Run failed: cannot unpack non-iterable NoneType object
Run failed: cannot unpack non-iterable NoneType object
Run failed: cannot unpack non-iterable NoneType object
